<a href="https://colab.research.google.com/github/YNWA-Algo/YNWA-Algo/blob/main/C_Bitcoin_5_year_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Install Crypto APIs/libs, plotly to create candle sticks charts

In [ ]:
!pip install pycoingecko
!pip install plotly
!pip install mplfinance

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 70 kB 3.3 MB/s 


In [ ]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go
from plotly.offline import plot
import matplotlib.pyplot as plt
import datetime
from pycoingecko import CoinGeckoAPI
from mplfinance.original_flavor import candlestick2_ohlc

Start off by getting the data we need. Using the get_coin_market_chart_by_id(id, vs_currency, days). id is the name of the coin i want - BITCOIN, vs_currency is the currency you want the price in (USD), and days is how many days back from today you want (1260 or 5 yrs)


In [ ]:
cg = CoinGeckoAPI()

bitcoin_data = cg.get_coin_market_chart_by_id(id='bitcoin', vs_currency='usd', days=1260)

In [ ]:
type(bitcoin_data )

dict

The response -> JSON which includes the price, market caps, and total volumes along with timestamps for each observation.  We are focused on the prices so we will select that data.


In [ ]:
bitcoin_price_data = bitcoin_data['prices']

bitcoin_price_data[0:5]

[[1548633600000, 3553.121681129778],
 [1548720000000, 3432.215668301249],
 [1548806400000, 3413.3754832497975],
 [1548892800000, 3457.8826491759864],
 [1548979200000, 3431.943637864456]]

Turn the data into a Pandas Dataframe

In [ ]:
data = pd.DataFrame(bitcoin_price_data, columns=['TimeStamp', 'Price'])

Now that we have the DataFrame we will convert the timestamp to datetime and save it as a column called 'Date'. We will map our unix_to_datetime to each timestamp and convert it to a readable datetime

In [ ]:
data['date'] = data['TimeStamp'].apply(lambda d: datetime.date.fromtimestamp(d/1000.0))

Using this modified dataset we can now group by the Date and find the min, max, open, and close for the candlesticks.


In [ ]:
candlestick_data = data.groupby(data.date, as_index=False).agg({"Price": ['min', 'max', 'first', 'last']})

Finally we are now ready to use plotly to create our Candlestick Chart.


In [ ]:
fig = go.Figure(data=[go.Candlestick(x=candlestick_data['date'],
                open=candlestick_data['Price']['first'], 
                high=candlestick_data['Price']['max'],
                low=candlestick_data['Price']['min'], 
                close=candlestick_data['Price']['last'])
                ])

fig.update_layout(xaxis_rangeslider_visible=False)

fig.show()